In [13]:
import cv2 
import pandas as pd
import numpy as np
import os
import random
from os import walk
from PIL import ImageFile
import seaborn as sns
import io
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_curve
#random.seed(42)
ImageFile.LOAD_TRUNCATED_IMAGES = True
# from DataGenetors import ImgDataParameters,DataGenerator
# from CAT_Net2 import CATNet2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras import optimizers
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
#The GPU id to use, usually either "0" or "1"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"
os.environ['OPENCV_IO_MAX_IMAGE_PIXELS']=str(2**96)
import sys
sys.path.append("..")
from WSI_Preprocessing.Preprocessing.WSI_Scanning import readWSI
from WSI_Preprocessing.Preprocessing.Denoising import denoising
from WSI_Preprocessing.Preprocessing.Patch_extraction_creatia import patch_extraction_random, all_patches_extarction
from WSI_Preprocessing.Preprocessing.Utilities import stainremover_small_patch_remover1
import openslide
from openslide import (OpenSlide, OpenSlideError,OpenSlideUnsupportedFormatError)
# from keract import get_activations
# from keract import display_activations

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)
def mean_act(y_true, y_pred):
    return K.mean(y_true)


In [14]:
model = load_model(r'/home/wsai/Experiment_F/Model_PN/new_modelct.h5')
layer_name = 'mixed10'
intermediate_layer_model = Model(inputs=model.input,
outputs=[model.get_layer(layer_name).output,model.output])

In [15]:
def plot(y_pred):
    plt = sns.heatmap(np.array([[y_pred]]), yticklabels = False,
                      xticklabels = False, cmap='coolwarm', 
                      vmin = 0, vmax = 1, cbar = False).get_figure()
    plt.savefig("example.png")
    img = cv2.imread("example.png")
    img  = np.where(img != [255,255,255],img , img[144,144])
    img = cv2.resize(img, (299,299))
    os.remove("example.png")
    return img

def ReLU(x):
    return x * (x > 0)


# layer_name = 'mixed10'
# intermediate_layer_model = Model(inputs=model.input,
# outputs=model.get_layer(layer_name).output)
# intermediate_output = intermediate_layer_model.predict(patchs1)
def extractingPatches(inputsvs, outputpath, magnification, patch_extraction_creatia = None,num_of_patches = 2000, 
                      filtering = "GaussianBlur", patch_size = (299,299), upperlimit = 900, 
                      lowerlimit = 300, red_value = (80,220), green_value = (80,200), blue_value = (80, 170),  
                      reconstructedimagepath = None, Annotation = None, Annotatedlevel = 0, Requiredlevel = 0, 
                      Requiredlevel1 = 0,model=None):
    
    Y= []
    slide1,slidedim = readWSI(inputsvs, "20x", Annotation, Annotatedlevel, Requiredlevel)
    Y_ = []
    Y_s = []
    k = 0 
    k1 = 0 
    ALL_P = []
    ALL_P1 = []
    all_patchs1 = []
#     patch_count = {'cancer' : 0, 'normal' : 0}
#     reconstrcutedimage = np.zeros_like(slide1)
    for i in range(int(len(slide1[0])/299)):
        for j in range(int(len(slide1)/299)):
            
            centerpoint20x = ((j*299+(patch_size[0]/2)),(i*299+(patch_size[1]/2)))
            sample_img = slide1[int(centerpoint20x[0]-patch_size[0]/2):int(centerpoint20x[0]+patch_size[0]/2),
                         int(centerpoint20x[1] - patch_size[1]/2): int(centerpoint20x[1] + patch_size[1]/2)]
            patchs = stainremover_small_patch_remover1(sample_img, patch_size)
            if patchs is None:
                None
            else:
                patchs1 = patchs/255
                patchs1 = np.expand_dims(patchs1, axis=0)
#                 y_pred = model.predict(patchs1)
                al_p = []
                al_p1 = []

#                 intermediate_output,y_pred = intermediate_layer_model.predict(patchs1)
#                  np.insert(np.array(intermediate_output),0,1,axis =1)
                y_pred = model.predict(patchs1)
#                 img = plot(y_pred[0][0])
#                 y = y_pred[0][0]
#                 all_patchs1.append(y)
#                 if y> 0.5: patch_count['cancer'] += 1
#                 else : patch_count['normal'] += 1
#                 al_20 = np.zeros((1,8,8))
#                 for t in range(len(intermediate_output[0][0][0])):
#                     grad = np.gradient(intermediate_output[:,:,:,t].flatten(),abs(1-y_pred[0][0]+0.00008))
#                     al = sum(grad)
#                     al_20 += al*intermediate_output[:,:,:,t]
#                 al_20 = ReLU(al_20)
#                 ALL_P.append(np.insert(np.array((sorted(al_20.flatten(), reverse=True))),0,y_pred[0][0]))
                
#                 ALL_P.append((sorted(al_20.flatten(), reverse=True)))
                ALL_P1.append((y_pred[0][0],j,i))
#                 reconstrcutedimage[int(centerpoint20x[0]-patch_size[0]/2):int(centerpoint20x[0]+patch_size[0]/2),
#                          int(centerpoint20x[1] - patch_size[1]/2): int(centerpoint20x[1] + patch_size[1]/2)] =  img
    return ALL_P1

In [16]:
from PIL import Image as im
import csv
path_mod = '/data/SharedStorage/SharedStorage1/LungCancerWSI/5-negative case//'
list_mod = os.listdir(path_mod)
list_slide = []

for i in list_mod:
    if i[-3:] == 'svs':
        list_slide.append(i)

dict_y_pred = []
dict_y_count = []
for i in list_slide[0:20]:
    tmp_pred = {}
    tmp_count = {}
    tmp_heatmap = {}
    print(i)
    data = path_mod + i
    slide = OpenSlide(data)
    slide_dimensions = slide.level_dimensions
    if len(slide_dimensions) == 3:
        ALL_P1= extractingPatches(data,"temp",magnification = "20x",patch_size= (299,299),Annotation = None,Annotatedlevel = 0, Requiredlevel = 0,Requiredlevel1 = 1,model=model)
    else:
        ALL_P1 = extractingPatches(data,"temp",magnification = "20x",patch_size= (299,299),Annotation = None,  Annotatedlevel = 0, Requiredlevel = 1,Requiredlevel1 = 2,model=model)
    gradcam_20_path1 = '/home/wsai//Newinter/PM/5-Normal/' + i[-8:-4] + "new"
#     gradcam_20_path = '/home/wsai/Newinter/H&E/Mod-diff/' + i[-11:-4]
#     all_patchs1 = np.sort(np.array(ALL_P), axis=0)[::-1]
    np.save(gradcam_20_path1, ALL_P1)
#     np.save(gradcam_20_path1, ALL_P1)
 

5-s12-551.svs
0
5-S-12-13682.svs
0
5-s12-2288.svs
0
5-s12-13683.svs
1
5-S-12-03099.svs
0
5-S-12-02988.svs
0
5-s12-03186.svs
0
5-s12-2521.svs
0
5-s12-4603.svs
0
5-s12-5874.svs
0
5-s12-7180.svs
0
5-s12-5840.svs
0
5-s12-8222.svs
0
5-s12-8593.svs
0
5-s12-9326.svs
0
5-s12-12951.svs
1
5-s12-16446.svs
0
5-s12-15749.svs
0
5-s12-16445.svs
0
5-s12-03882.svs
0
